![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_DistilBertForQuestionAnswering.ipynb)

# Import OpenVINO DistilBertForQuestionAnswering  models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting DistilBertForQuestionAnswering  models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for DistilBertForQuestionAnswering from DistilBertForQuestionAnswering and they have to be in `Question Answering` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00


[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [distilbert/distilbert-base-cased-distilled-squad](https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad) model from HuggingFace as an example and load it as a `OVModelForQuestionAnswering`, representing an OpenVINO model.
- In addition to the OVModelForQuestionAnswering model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import DistilBertTokenizer
from optimum.intel import OVModelForQuestionAnswering

MODEL_NAME = "distilbert/distilbert-base-cased-distilled-squad"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ort_model = OVModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

The model distilbert/distilbert-base-cased-distilled-squad was already converted to the OpenVINO IR but got `export=True`, the model will be converted to OpenVINO once again. Don't forget to save the resulting model with `.save_pretrained()`


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('ov_models/distilbert/distilbert-base-cased-distilled-squad/tokenizer_config.json',
 'ov_models/distilbert/distilbert-base-cased-distilled-squad/special_tokens_map.json',
 'ov_models/distilbert/distilbert-base-cased-distilled-squad/vocab.txt',
 'ov_models/distilbert/distilbert-base-cased-distilled-squad/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [9]:
!ls -l {EXPORT_PATH}

total 255084
-rw-r--r-- 1 root root       580 Jun 21 19:56 config.json
-rw-r--r-- 1 root root 260774048 Jun 21 19:56 openvino_model.bin
-rw-r--r-- 1 root root    197840 Jun 21 19:56 openvino_model.xml
-rw-r--r-- 1 root root       125 Jun 21 19:56 special_tokens_map.json
-rw-r--r-- 1 root root      1279 Jun 21 19:56 tokenizer_config.json
-rw-r--r-- 1 root root    213450 Jun 21 19:56 vocab.txt


As you can see, we need to move `vocab.txt` from the tokenizer to `assets` folder which Spark NLP will look for

In [10]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets

In [11]:
!ls -lR {EXPORT_PATH}

ov_models/distilbert/distilbert-base-cased-distilled-squad:
total 254876
drwxr-xr-x 2 root root      4096 Jun 21 20:32 assets
-rw-r--r-- 1 root root       580 Jun 21 19:56 config.json
-rw-r--r-- 1 root root 260774048 Jun 21 19:56 openvino_model.bin
-rw-r--r-- 1 root root    197840 Jun 21 19:56 openvino_model.xml
-rw-r--r-- 1 root root       125 Jun 21 19:56 special_tokens_map.json
-rw-r--r-- 1 root root      1279 Jun 21 19:56 tokenizer_config.json

ov_models/distilbert/distilbert-base-cased-distilled-squad/assets:
total 212
-rw-r--r-- 1 root root 213450 Jun 21 19:56 vocab.txt


Voila! We have our `vocab.txt` inside assets directory

## Import and Save DistilBertForQuestionAnswering in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 19.6 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `DistilBertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [12]:
from sparknlp.annotator import DistilBertForQuestionAnswering

spanClassifier = DistilBertForQuestionAnswering.loadSavedModel(
     EXPORT_PATH,
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [13]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [14]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your DistlBertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [15]:
! ls -l {EXPORT_PATH}_spark_nlp_openvino

total 254908
-rw-r--r-- 1 root root 261011996 Jun 21 20:33 distilbert_classification_openvino
drwxr-xr-x 3 root root      4096 Jun 21 20:33 fields
drwxr-xr-x 2 root root      4096 Jun 21 20:33 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [16]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import DistilBertForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = DistilBertForQuestionAnswering.load("./{}_spark_nlp_openvino".format(EXPORT_PATH))\
    .setInputCols(["document_question", "document_context"])\
    .setOutputCol("answer")

pipeline = Pipeline(stages=[
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([
    ["What's my name?", "My name is Clara and I live in Berkeley."]
]).toDF("question", "context")

model = pipeline.fit(example)
result = model.transform(example)

result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[Clara]|
+-------+



That's it! You can now go wild and use hundreds of `DistilBertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
